# easy_ViTPose demo
Be sure to enable GPU runtime

First time you run it all need to restart the runtime and rerun

In [ ]:
!git clone https://github.com/JunkyByte/easy_ViTPose.git
!cd easy_ViTPose/ && pip install -r requirements.txt && pip install -e .
!pip install huggingface_hub

fatal: destination path 'easy_ViTPose' already exists and is not an empty directory.
Obtaining file:///content/easy_ViTPose
  Preparing metadata (setup.py) ... done
  Attempting uninstall: easy_ViTPose
    Found existing installation: easy_ViTPose 1.1
    Uninstalling easy_ViTPose-1.1:
      Successfully uninstalled easy_ViTPose-1.1
  Running setup.py develop for easy_ViTPose


# Restart the runtime (runtime -> restart runtime) to update installed packages versions

# Download the models

In [ ]:
#@title Choose model and run this cell

MODEL_SIZE = 'b'  #@param ['s', 'b', 'l', 'h']
YOLO_SIZE = 's'  #@param ['s', 'n']
DATASET = 'wholebody'  #@param ['coco_25', 'coco', 'wholebody', 'mpii', 'aic', 'ap10k', 'apt36k']
ext = '.pth'
ext_yolo = '.pt'

In [ ]:
import os
from huggingface_hub import hf_hub_download
MODEL_TYPE = "torch"
YOLO_TYPE = "torch"
REPO_ID = 'JunkyByte/easy_ViTPose'
FILENAME = os.path.join(MODEL_TYPE, f'{DATASET}/vitpose-' + MODEL_SIZE + f'-{DATASET}') + ext
FILENAME_YOLO = 'yolov8/yolov8' + YOLO_SIZE + ext_yolo

print(f'Downloading model {REPO_ID}/{FILENAME}')
model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
yolo_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME_YOLO)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Run inference

In [ ]:
# If you get an error from PIL restart environment and rerun this cell to update packages version
from easy_ViTPose import VitInference
model = VitInference(model_path, yolo_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=320, is_video=False)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
#@title Video inference
from google.colab import files
import numpy as np
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt
from vit_utils.visualization import draw_points_and_skeleton, joints_dict

import argparse
import json
import os
import time
import glob

import cv2
import torch
import tqdm

from easy_ViTPose.vit_utils.inference import NumpyEncoder, VideoReader
from easy_ViTPose.inference import VitInference
from easy_ViTPose.vit_utils.visualization import joints_dict


SHOW = True
SAVE_IMG = True

input_path = "62794_6.mp4"
output_path_json = "out.json"


model = VitInference(model_path, yolo_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=320, is_video=True)

def do_inference(input_path, save_img=False):
    reader = VideoReader(input_path, 0)
    cap = cv2.VideoCapture(input_path)  # type: ignore
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    wait = 15
    if SAVE_IMG:
        cap = cv2.VideoCapture(input_path)  # type: ignore
        fps = cap.get(cv2.CAP_PROP_FPS)
        ret, frame = cap.read()
        cap.release()
        assert ret
        assert fps > 0
        output_size = frame.shape[:2][::-1]

        # Check if we have X264 otherwise use default MJPG
        try:
            temp_video = cv2.VideoWriter('/tmp/checkcodec.mp4',
                                          cv2.VideoWriter_fourcc(*'h264'), 30, (32, 32))
            opened = temp_video.isOpened()
        except Exception:
            opened = False
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # More compatible than h264 or MJPG
        out_writer = cv2.VideoWriter("out.mp4", fourcc, fps, output_size)

        # Check if it opened correctly
        if not out_writer.isOpened():
            raise RuntimeError("Failed to open VideoWriter. Check codec and path.")



    print(f'>>> Running inference on {input_path}')
    keypoints = []
    fps = []
    tot_time = 0.
    for (ith, img) in tqdm.tqdm(enumerate(reader), total=total_frames):
        t0 = time.time()

        # Run inference
        frame_keypoints = model.inference(img)
        keypoints.append(frame_keypoints)

        delta = time.time() - t0
        tot_time += delta
        fps.append(delta)

        # Draw the poses and save the output img
        if SHOW or SAVE_IMG:
            # Draw result and transform to BGR
            img = model.draw(True, True, 0.5)[..., ::-1]
            if SAVE_IMG:
                out_writer.write(img)

    tot_poses = sum(len(k) for k in keypoints)
    print(f'>>> Mean inference FPS: {1 / np.mean(fps):.2f}')
    print(f'>>> Total poses predicted: {tot_poses} mean per frame: '
          f'{(tot_poses / (ith + 1)):.2f}')
    print(f'>>> Mean FPS per pose: {(tot_poses / tot_time):.2f}')

    print('>>> Saving output json')
    with open(output_path_json, 'w') as f:
        out = {'keypoints': keypoints,
                'skeleton': joints_dict()[model.dataset]['keypoints']}
        json.dump(out, f, cls=NumpyEncoder)

    if SAVE_IMG:
        out_writer.release()
    cv2.destroyAllWindows()
    model.reset()

'''
frame_keypoints = model.inference(img)
img = model.draw(show_yolo=False)

from google.colab.patches import cv2_imshow
cv2_imshow(img[..., ::-1])
'''

'\nframe_keypoints = model.inference(img)\nimg = model.draw(show_yolo=False)\n\nfrom google.colab.patches import cv2_imshow\ncv2_imshow(img[..., ::-1])\n'

In [ ]:
#@title Estimate poses for all videos

from google.colab import drive
import os

drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive/DL_data/'
video_dir = data_dir + 'ohp_labeled/correct/'
video_files = os.listdir(video_dir)

out_dir = data_dir + 'ohp_poses/correct/'
if not os.path.exists(out_dir):
  os.makedirs(out_dir)

i=0
for filename in video_files:
    i += 1
    print(f"estimating video {i}")

    filepath = video_dir + filename
    output_path_json = out_dir + filename[:-4] + '.json'

    if os.path.exists(output_path_json):
      print("file already exists")
      continue

    do_inference(filepath)

drive.flush_and_unmount()

Mounted at /content/drive
estimating video 1
file already exists
estimating video 2
file already exists
estimating video 3
file already exists
estimating video 4
file already exists
estimating video 5
file already exists
estimating video 6
file already exists
estimating video 7
file already exists
estimating video 8
file already exists
estimating video 9
file already exists
estimating video 10
file already exists
estimating video 11
file already exists
estimating video 12
file already exists
estimating video 13
file already exists
estimating video 14
file already exists
estimating video 15
file already exists
estimating video 16
file already exists
estimating video 17
file already exists
estimating video 18
file already exists
estimating video 19
file already exists
estimating video 20
file already exists
estimating video 21
file already exists
estimating video 22
file already exists
estimating video 23
file already exists
estimating video 24
file already exists
estimating video 25
fil

100%|██████████| 58/58 [01:31<00:00,  1.58s/it]


>>> Mean inference FPS: 0.64
>>> Total poses predicted: 98 mean per frame: 1.69
>>> Mean FPS per pose: 1.08
>>> Saving output json
estimating video 1002
>>> Running inference on /content/drive/MyDrive/DL_data/ohp_labeled/correct/62824_1.mp4


100%|██████████| 162/162 [04:01<00:00,  1.49s/it]


>>> Mean inference FPS: 0.68
>>> Total poses predicted: 249 mean per frame: 1.54
>>> Mean FPS per pose: 1.05
>>> Saving output json
estimating video 1003
>>> Running inference on /content/drive/MyDrive/DL_data/ohp_labeled/correct/62943_6.mp4


...

>>> Mean inference FPS: 0.97
>>> Total poses predicted: 205 mean per frame: 1.05
>>> Mean FPS per pose: 1.02
>>> Saving output json
estimating video 1017
file already exists
estimating video 1018
>>> Running inference on /content/drive/MyDrive/DL_data/ohp_labeled/correct/63252_1.mp4


100%|██████████| 327/327 [05:33<00:00,  1.02s/it]


>>> Mean inference FPS: 1.01
>>> Total poses predicted: 327 mean per frame: 1.00
>>> Mean FPS per pose: 1.01
>>> Saving output json
estimating video 1019
file already exists
estimating video 1020
file already exists
estimating video 1021
file already exists
estimating video 1022
file already exists
estimating video 1023
file already exists
estimating video 1024
>>> Running inference on /content/drive/MyDrive/DL_data/ohp_labeled/correct/63308_6.mp4


100%|██████████| 113/113 [01:59<00:00,  1.06s/it]


>>> Mean inference FPS: 0.97
>>> Total poses predicted: 113 mean per frame: 1.00
>>> Mean FPS per pose: 0.97
>>> Saving output json
